## 글로브(Global Vectors for Word Representation, GloVe)

카운트 기반과 예측 기반을 모두 사용하는 방법론

카운트 기반의 LSA(Latent Semantic Analysis)와 예측 기반의 Word2Vec의 단점을 보안한다는 목적으로 나옴

Word2Vec만큼 뛰어난 성능

Word2Vec와 GloVe 중에서 어떤 것이 더 뛰어나진 않음, 사용해보고 성능이 더 좋은 것을 사용

LSA는 각 단어의 빈도수를 카운트 한 행렬이라는 전체적인 통계 정보를 입력으로 받아 차원을 축소(Truncated SVD)하여 잠재된 의미를 끌어내는 방법론

카운트 기반으로 코퍼스의 전체적인 통계 정보를 고려하기는 하지만 단어 의미의 유추 작업(Analogy task)에는 성능이 떨어짐

Word2Vec는 실제값과 예측값에 대한 오차를 손실 함수를 통해 줄여나가며 학습하는 예측 기반의 방법론

예측 기반으로 단어 간 유추 작업에는 LSA보다 뛰어나지만 임베딩 벡터가 윈도우 크기 내에서만 주변 단어를 고려 코퍼스의 전체적인 통계 정보를 반영하지 않음

### 윈도우 기반 동시 등장 행렬(Window based Co-occurrence Matrix)

단어의 동시 등장 행렬은 행과 열을 전체 단어 집합의 단어들로 구성하고, 

i 단어의 윈도우 크기(Window Size) 내에서 k 단어가 등장한 횟수를 i행 k열에 기재한 행렬

윈도우 크기가 N일 때는 좌, 우에 존재하는 N개의 단어만 참고

해당 행렬은 전치(Transpose)해도 동일한 행렬이 된다는 특징

i 단어의 윈도우 크기 내에서 k 단어가 등장한 빈도는 반대로 k 단어의 윈도우 크기 내에서 i 단어가 등장한 빈도와 동일하기 때문

### 동시 등장 확률(Co-occurrence Probability)

등장 확률(Co-occurrence Probability) $P(k|i)$ 는 동시 등장 행렬로부터 특정 단어 i의 전체 등장 횟수를 카운트하고

특정 단어 i가 등장했을 때 어떤 단어 k가 등장한 횟수를 카운트하여 계산한 조건부 확률

i를 중심 단어(Center Word), k를 주변 단어(Context Word)라고 했을 때

동시 등장 행렬에서 중심 단어 i의 행의 모든 값을 더한 값을 분모로 하고 i행 k열의 값을 분자로 한 값

### GloVe 손실 함수(Loss function)

$X$ : 동시 등장 행렬(Co-occurrence Matrix)

$X_{ij}$ : 중심 단어 i가 등장했을 때 윈도우 내 주변 단어 j가 등장하는 횟수

$X_i : \sum_j X_{ij}$ : 동시 등장 행렬에서 i행의 값을 모두 더한 값

$P_{ik} : P(k|i) = \frac{X_{ik}}{X_i}$ : 중심 단어 i가 등장했을 때 윈도우 내 주변 단어 k가 등장할 확률

$\frac{P_{ik}}{P_{jk}} : P_{ik}$ 를 $P_{jk}$ 로 나눠준 값

$w_i$ : 중심 단어 i의 임베딩 벡터

$\tilde{w_k}$ : 주변 단어 k의 임베딩 벡터

임베딩 된 중심 단어와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 확률이 되도록 만드는 것

$ \text{dot product}(w_i, \tilde{w_k}) \approx P(k|i) = P_{ik} $

GloVe는 log를 넣어 아래와 같은 관계를 가지도록 임베딩 벡터를 설계

$ \text{dot product}(w_i, \tilde{w_k}) \approx log \, P(k|i) = log \, P_{ik} $

벡터 $ w_i, w_j, \tilde{w_k} $ 를 가지고 어떤 함수 $F$를 수행하면 $ \frac{P_{ik}}{P_{jk}} $ 초기식으로 전개

$ F(w_i, w_j, \tilde{w_k}) = \frac{P_{ik}}{P_{jk}} $

함수 F는 두 단어 사이의 동시 등장 확률의 크기 관계 비(ratio) 정보를 벡터 공간에 인코딩하는 것이 목적

따라서 벡터 $ w_i, w_j $의 차를 입력으로 사용하도록 변경

$ F(w_i - w_j, \tilde{w_k}) = \frac{P_{ik}}{P_{jk}} $

이때 우변은 스칼라값이고 좌변은 벡터값, 이를 성립하기 해주기 위해서 함수 F의 두 입력에 내적(Dot product)을 수행

$ F((w_i - w_j)^T \tilde{w_k}) = \frac{P_{ik}}{P_{jk}} $

선형 공간(Linear space)에서 단어의 의미 관계를 표현하기 위해 뺄셈과 내적을 수행

중심단어와 주변단어의 선택 기준은 실제로는 무작위 선택, 둘의 관계는 자유롭게 교환될 수 있도록 해야함

함수 F가 실수의 덧셈과 양수의 곱셈에 대해서 준동형(Homomorphism)을 만족하도록 해야함

즉 F(a+b)가 F(a)F(b)를 만족시켜야 함, 수식으로 나타내면

$ F(a + b) = F(a)F(b), \forall a, b \in R $

함수 F는 결과값으로 스칼라 값이 나와야 함

준동형식에서 a와 b가 각각 벡터값이라면 함수 F의 결과값으로는 스칼라 값이 나올 수 없지만, a와 b가 각각 사실 두 벡터의 내적값이라고 하면 결과값으로 스칼라 값이 나올 수 있음

V를 벡터 v를 벡터값이라 할때 다음과 같이 식을 바꿀 수 있음

$ F(v_1^T v_2 + v_3^T v_4) = F(v_1^T v_2)F(v_3^T v_4), \forall v_1, v_2, v_3, v_4 \in V $

앞서 작성한 GloVe 식에서는 $w_i, w_j$ 두 벡터의 차이를 함수의 입력으로 사용 준동형 식을 이를 뺄셈에 대한 준동형식으로 변경

$ F(v_1^T v_2 - v_3^T v_4) = \frac{F(v_1^T v_2)}{F(v_3^T v_4)}, \forall v_1, v_2, v_3, v_4 \in V $

이 준동형 식을 GloVe 식에 적용

$ F((w_i - w_j)^T \tilde{w_k}) = \frac{F(w_i^T \tilde{w_k})}{F(w_j^T \tilde{w_k})} $

이전식의 우변은 원래 $\frac{P_{ik}}{P_{jk}}$ 였으므로 결과적으론 다음과 같음

$ \frac{P_{ik}}{P_{jk}} = \frac{F(w_i^T \tilde{w_k})}{F(w_j^T \tilde{w_k})} $

$ F(w_i^T \tilde{w_k}) = P_{ik} = \frac{X_{ik}}{X_i} $

$ F(w_i^T \tilde{w_k} - w_j^T \tilde{w_k}) = \frac{F(w_i^T \tilde{w_k})}{F(w_j^T \tilde{w_k})} $

이는 뺄셈에 대한 준동형식의 형태와 정확히 일치

이제 이를 만족하는 함수 F를 찾아야 함, 이를 만족하는 함수는 지수 함수(Exponential function), F를 exp라 하면

$ exp(w_i^T \tilde{w_k} - w_j^T \tilde{w_k}) = \frac{exp(w_i^T \tilde{w_k})}{exp(w_j^T \tilde{w_k})} $

$ exp(w_i^T \tilde{w_k}) = P_{ik} = \frac{X_{ik}}{X_i} $

두번째 식으로부터 다음과 같은 식을 얻을 수 있음

$ w_i^T \tilde{w_k} = log \, P_{ik} = log \, (\frac{X_{ik}}{X_i}) = log \, X_{ik} - log \, X_i $

$w_i, \tilde{w_k}$ 는 두 값의 위치를 서로 바꾸어도 식이 성립해야 함

$log \, X_i$ 이 없으면 이를 성립시킬 수 있음, 따라서 이 항을 $w_i$에 대한 편향 $b_i$라는 상수항으로 대체, 같은 이유로 $\tilde{w_k}$도 편향 $\tilde{b_k}$를 추가

$ w_i^T \tilde{w_k} + b_i + \tilde{b_k} = log \, X_{ik} $

이 식이 손실 함수의 핵심이 되는 식

우변의 값과의 차이를 최소화는 방향으로 좌변의 4개의 항은 학습을 통해 값이 바뀌는 변수들이 됨

다음과 같이 손실 함수를 일반화 하면

$ Loss \, function = \displaystyle\sum_{m,n=1}^{V} (w_m^T \tilde{w_n} + b_m + \tilde{b_n} - log \, X_{mn})^2 $

V는 단어 집합 크기를 의미, 아직 문제점이 있음

$log \, X_{ik}$는 $X_{ik}$ 값이 0이 될 수 있음, X는 DTM처럼 희소 행렬(Sparse Matrix)일 가능성이 높음

동시 등장 행렬에서 동시 등장 빈도의 값 $X_{ik}$가 굉장히 낮은 경우에는 정보에 거의 도움이 되지 않는다고 판단

GloVe는 $X_{ik}$의 값에 영향을 받는 가중치 함수(Weighting function) $f(X_{ik})$를 손실 함수에 도입

$X_{ik}$ 의 값이 작으면 상대적으로 함수의 값은 작도록 하고, 값이 크면 함수의 값은 상대적으로 크도록함

지나친 가중치를 주지 않기위해서 함수의 최대값이 1로 정해짐, 해당 함수 f(x)식은 다음과 같음

$ f(x) = min(1, (x/x_{max})^{3/4}) $

최종적으로 일반화 된 손실 함수는 다음과 같음

$ Loss \, function = \displaystyle\sum_{m,n=1}^{V} f(X_{mn})(w_m^T \tilde{w_n} + b_m + \tilde{b_n} - log \, X_{mn})^2 $

In [3]:
import numpy as np
import scipy.sparse as sp

In [4]:
class GloVe:
    def __init__(self, vocab_size, embedding_dim, x_max=100, alpha=0.75, learning_rate=0.05):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.x_max = x_max
        self.alpha = alpha
        self.learning_rate = learning_rate
        self.W = np.random.randn(vocab_size, embedding_dim)
        self.W_tilde = np.random.randn(vocab_size, embedding_dim)
        self.b = np.random.randn(vocab_size)
        self.b_tilde = np.random.randn(vocab_size)
        self.gradsq_W = np.ones((vocab_size, embedding_dim))
        self.gradsq_W_tilde = np.ones((vocab_size, embedding_dim))
        self.gradsq_b = np.ones(vocab_size)
        self.gradsq_b_tilde = np.ones(vocab_size)
    
    def fit(self, cooccurrence_matrix, epochs=50):
        non_zero_elements = sp.find(cooccurrence_matrix)
        for epoch in range(epochs):
            total_cost = 0
            for i, j, X_ij in zip(*non_zero_elements):
                weight = min(1.0, (X_ij / self.x_max)**self.alpha)
                cost = (self.W[i, :].dot(self.W_tilde[j, :]) + self.b[i] + self.b_tilde[j] - np.log(X_ij))
                total_cost += 0.5 * weight * (cost**2)
                
                grad_W_i = weight * cost * self.W_tilde[j, :]
                grad_W_tilde_j = weight * cost * self.W[i, :]
                grad_b_i = weight * cost
                grad_b_tilde_j = weight * cost
                
                self.W[i, :] -= self.learning_rate * grad_W_i / np.sqrt(self.gradsq_W[i, :])
                self.W_tilde[j, :] -= self.learning_rate * grad_W_tilde_j / np.sqrt(self.gradsq_W_tilde[j, :])
                self.b[i] -= self.learning_rate * grad_b_i / np.sqrt(self.gradsq_b[i])
                self.b_tilde[j] -= self.learning_rate * grad_b_tilde_j / np.sqrt(self.gradsq_b_tilde[j])
                
                self.gradsq_W[i, :] += grad_W_i**2
                self.gradsq_W_tilde[j, :] += grad_W_tilde_j**2
                self.gradsq_b[i] += grad_b_i**2
                self.gradsq_b_tilde[j] += grad_b_tilde_j**2
                
            print(f"Epoch {epoch+1}, Cost: {total_cost}")